<a href="https://colab.research.google.com/github/ftk1000/torch_demos/blob/master/torch_gpu_comp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Compare runtime with GPU and w/o GPU

In [2]:
import torch
import time

if torch.cuda.is_available():
  device = torch.device("cuda")  # Edit -> Notebook Settings -> Hardware Accelerator = GPU
else:
  device = torch.device("cpu")

print(device)

cuda


In [13]:
# Get data
N_rows = 20000
N_features = 8
X_train = torch.randn(  int(N_rows*0.8) , N_features,  device=device)
Y_train = torch.randn(  int(N_rows*0.8) , 1, device=device)
X_test = torch.randn(  int(N_rows*0.2) , N_features,  device=device)
Y_test = torch.randn(  int(N_rows*0.2) , 1, device=device)
print(X_train.size(), Y_train.size(), X_test.size(), Y_test.size())

torch.Size([16000, 8]) torch.Size([16000, 1]) torch.Size([4000, 8]) torch.Size([4000, 1])


In [4]:
print(X_train.size())
print(Y_train.size())
print(X_train[:3])
print(Y_train[:3])

torch.Size([16000, 8])
torch.Size([16000, 1])
tensor([[-0.3686,  0.2725,  0.2180,  0.2388, -0.8597,  0.8967,  1.5696, -1.1602],
        [-0.9024,  0.0794,  2.1494, -0.0179, -0.1237,  1.5971, -0.4397, -1.5321],
        [ 1.3780, -1.6885,  0.3785, -0.9023,  0.9839, -0.9624, -1.4750,  0.4680]],
       device='cuda:0')
tensor([[ 0.1565],
        [-0.1178],
        [ 0.5565]], device='cuda:0')


In [30]:
# import torch
from torch.autograd import Variable
# import torch.nn.functional as F
import torch.utils.data as Data

# D_in is input dimension; H is hidden dimension; D_out is output dimension.
D_in = N_rows
H=100 
D_out = 1

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
).to(device)

print(D_in, H, D_out)

2000 100 1


In [5]:
# torch.mean(y) #tensor(-109.9649, device='cuda:0')
# import torch
from torch.autograd import Variable
# import torch.nn.functional as F
import torch.utils.data as Data

In [6]:
N    = X_train.size()[0]
D_in = X_train.size()[1]
H    = 64
D_out = 1
print(N,D_in,H,D_out)

16000 8 64 1


In [7]:
# Define model 
# ref: https://medium.com/@benjamin.phillips22/simple-regression-with-neural-networks-in-pytorch-313f06910379

# model = torch.nn.Sequential(
#         torch.nn.Linear(D_in, 200),
#         torch.nn.LeakyReLU(),
#         torch.nn.Linear(200, 100),
#         torch.nn.LeakyReLU(),
#         torch.nn.Linear(100, 1),
#     ).to(device)

model = torch.nn.Sequential(
        torch.nn.Linear(D_in, 200),
        torch.nn.LeakyReLU(),
        torch.nn.Linear(200, 100),
        torch.nn.LeakyReLU(),
        torch.nn.Linear(100, 50),
        torch.nn.LeakyReLU(),
        torch.nn.Linear(50, 1),
    ).to(device)


optimizer = torch.optim.Adam(model.parameters(), lr=0.01)        #.to(cuda_device)
loss_func = torch.nn.MSELoss().to(device)  # this is for regression mean squared loss

BATCH_SIZE = 64
EPOCH = 200

In [8]:
# https://datascience.stackexchange.com/questions/45916/loading-own-train-data-and-labels-in-dataloader-using-pytorch
train_data = []
for i in range(len(X_train)):
   train_data.append([X_train[i], Y_train[i]])

trainloader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=64)
i1, l1= next(iter(trainloader))
print(i1.shape, l1.shape)

torch.Size([64, 8]) torch.Size([64, 1])


In [9]:
model(i1).size()

torch.Size([64, 1])

In [10]:
# trainloader = torch.utils.data.DataLoader(X_train, shuffle=True, batch_size=64)
# i1 = next(iter(trainloader))
# print(i1.shape)

i=0
for x,y in enumerate(trainloader):
    if (i==0):
        print(x)
        i+=1
    

0


In [11]:
trainloader.dataset[0]

[tensor([-0.3686,  0.2725,  0.2180,  0.2388, -0.8597,  0.8967,  1.5696, -1.1602],
        device='cuda:0'), tensor([0.1565], device='cuda:0')]

In [15]:
# start training
import time
EPOCH=50
SHOW_COUNT = 10
PERIOD = int(EPOCH/SHOW_COUNT)

rmses = []
start_time = time.time()
for epoch in range(EPOCH):
#     for step, (batch_x, batch_y) in enumerate(loader): # for each training step
    for step, (batch_x, batch_y) in enumerate(trainloader): # for each training step
        
        if epoch % PERIOD == PERIOD-1 and step==1:
            print("epoch step loss=",epoch, step, loss )
        
        b_x = Variable(batch_x)
        b_y = Variable(batch_y)

        prediction = model(b_x)     # input x and predict based on x
        torch.cuda.synchronize()

        loss = loss_func(prediction, b_y)     # must be (1. nn output, 2. target)
        rmses.append(float(loss)**0.5)

        optimizer.zero_grad()   # clear gradients for next train
        torch.cuda.synchronize()
        
        loss.backward()         # backpropagation, compute gradients
        torch.cuda.synchronize()
        
        optimizer.step()        # apply gradients
        torch.cuda.synchronize()

        
end_time = time.time()
train_time = end_time - start_time
print('train_time = ', train_time)

pred = model(X_test)     # input x and predict based on x
torch.sum((Y_test - pred)**2/Y_test.size()[0])**(1/2)

epoch step loss= 4 1 tensor(0.8452, device='cuda:0', grad_fn=<MseLossBackward>)
epoch step loss= 9 1 tensor(1.3218, device='cuda:0', grad_fn=<MseLossBackward>)
epoch step loss= 14 1 tensor(0.7527, device='cuda:0', grad_fn=<MseLossBackward>)
epoch step loss= 19 1 tensor(1.0772, device='cuda:0', grad_fn=<MseLossBackward>)
epoch step loss= 24 1 tensor(0.9458, device='cuda:0', grad_fn=<MseLossBackward>)
epoch step loss= 29 1 tensor(1.2990, device='cuda:0', grad_fn=<MseLossBackward>)
epoch step loss= 34 1 tensor(1.0450, device='cuda:0', grad_fn=<MseLossBackward>)
epoch step loss= 39 1 tensor(1.0354, device='cuda:0', grad_fn=<MseLossBackward>)
epoch step loss= 44 1 tensor(0.9591, device='cuda:0', grad_fn=<MseLossBackward>)
epoch step loss= 49 1 tensor(1.4052, device='cuda:0', grad_fn=<MseLossBackward>)
train_time =  44.793843269348145


tensor(1.0169, device='cuda:0', grad_fn=<PowBackward0>)

In [14]:
print('train_time = ', train_time)

pred = model(X_test)     # input x and predict based on x
torch.sum((Y_test - pred)**2/Y_test.size()[0])**(1/2)

train_time =  44.1050808429718


tensor(1.0176, device='cuda:0', grad_fn=<PowBackward0>)

In [31]:
x.size()

torch.Size([5])

In [32]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)        #.to(cuda_device)
loss_func = torch.nn.MSELoss().to(device)  # this is for regression mean squared loss

BATCH_SIZE = 64
EPOCH = 20

In [33]:
# https://datascience.stackexchange.com/questions/45916/loading-own-train-data-and-labels-in-dataloader-using-pytorch
train_data = []
for i in range(len(X_train)):
   train_data.append([X_train[i], Y_train[i]])

trainloader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=64)
# i1, l1= next(iter(trainloader))
# print(i1.shape, l1.shape)

In [34]:
# i=0
# for x,y in enumerate(trainloader):
#     if (i==0):
#         print(x,y)
#         i+=1

In [35]:
x,y=trainloader.dataset[0]
x = Variable(x)
print(x)
print(y)

tensor([ 0.8002, -1.7215,  0.3216, -1.5974,  0.7343, -1.8192, -0.2238,  0.6314],
       device='cuda:0')
tensor([0.8008], device='cuda:0')


In [36]:
px = model(x)
print(px)

RuntimeError: ignored

In [19]:
# start training
import time
EPOCH=50
SHOW_COUNT = 10
PERIOD = int(EPOCH/SHOW_COUNT)

rmses = []
start_time = time.time()
for epoch in range(EPOCH):
    for step, (batch_x, batch_y) in enumerate(trainloader): # for each training step
        
        if epoch % PERIOD == PERIOD-1 and step==1:
            print("epoch step loss=",epoch, step, loss )
        
        b_x = Variable(batch_x)
        b_y = Variable(batch_y)

        prediction = model(b_x)     # input x and predict based on x
        torch.cuda.synchronize()

        loss = loss_func(prediction, b_y)     # must be (1. nn output, 2. target)
        rmses.append(float(loss)**0.5)

        optimizer.zero_grad()   # clear gradients for next train
        torch.cuda.synchronize()
        
        loss.backward()         # backpropagation, compute gradients
        torch.cuda.synchronize()
        
        optimizer.step()        # apply gradients
        torch.cuda.synchronize()

        
end_time = time.time()
train_time = end_time - start_time


RuntimeError: ignored

In [ ]:
print('train_time = ', train_time)

pred = model(X_test)     # input x and predict based on x
torch.sum((Y_test - pred)**2/Y_test.size()[0])**(1/2)